In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install bnlp-toolkit
!pip install transformers
!pip install git+https://github.com/csebuetnlp/normalizer

In [ ]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader, TensorDataset
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AutoTokenizer, AutoModelForSequenceClassification, PreTrainedTokenizerFast
from normalizer import normalize

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

from bnlp import NLTKTokenizer
import math
import random

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
rand_seed = 1601
np.random.seed(rand_seed)
random.seed(rand_seed)
torch.manual_seed(rand_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(rand_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
df_train = pd.read_excel('/content/drive/Shareddrives/TFQ/SentNoB_After_Def/noise_train.xlsx')
df_test = pd.read_excel('/content/drive/Shareddrives/TFQ/SentNoB_After_Def/noise_test.xlsx')
df_val = pd.read_excel('/content/drive/Shareddrives/TFQ/SentNoB_After_Def/noise_val.xlsx')

In [ ]:
col_name = ['Slang', 'Local', 'Word Misuse', 'Context/Word Missing', 'Wrong Serial',
            'Mixed Language', 'Punctuation', 'Space Error', 'Spelling', 'Internet Garbage']

for col in col_name:
    df_train[col] = pd.to_numeric(df_train[col], errors='coerce')
    df_test[col] = pd.to_numeric(df_test[col], errors='coerce')
    df_val[col] = pd.to_numeric(df_val[col], errors='coerce')

df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_val = df_val.fillna(0)

In [ ]:
df_train.head(5)

,0,Data,Slang,Local,Word Misuse,Context/Word Missing,Wrong Serial,Mixed Language,Punctuation,Space Error,Spelling,Internet Garbage
0,0,মুগ্ধ হয়ে গেলাম মামু. আর তোমায় কি কমু. বলো তোম...,0.0,1,0,0,0,0,1.0,0,0.0,0
1,1,এই কুত্তার বাচ্চাদের জন্য দেশটা আজ এমন অবস্তায়...,1.0,0,0,0,0,0,1.0,0,1.0,0
2,2,ভাই আপনার কথাই যাদু রয়েছে,0.0,0,1,0,0,0,0.0,0,0.0,0
3,3,উওরটা আমার অনেক ভাল লেগেছে,0.0,0,1,0,0,0,0.0,0,1.0,0
4,4,আমার নিজের গাড়ী নিয়ে কি সাজেক যেতে পারবো না ?...,0.0,0,0,0,0,1,0.0,0,0.0,0


In [ ]:
X_train = df_train['Data'].values
y_train = df_train[col_name].values

X_test = df_test['Data'].values
y_test = df_test[col_name].values

X_val = df_val['Data'].values
y_val = df_val[col_name].values

In [ ]:
bnltk = NLTKTokenizer()

X_train_tokens = [bnltk.word_tokenize(str(sentence)) for sentence in X_train]
X_test_tokens = [bnltk.word_tokenize(str(sentence)) for sentence in X_test]
X_val_tokens = [bnltk.word_tokenize(str(sentence)) for sentence in X_val]

In [ ]:
words = []

for i in X_train_tokens:
  words.extend(i)

for i in X_test_tokens:
  words.extend(i)

for i in X_val_tokens:
  words.extend(i)

In [ ]:
from collections import Counter

In [ ]:
max_size = 110

In [ ]:
word_freq = Counter(words)
min_freq = 2
vocab = {}
for idx, tok_freq in enumerate(word_freq.items()):
  tok, freq = tok_freq
  if freq >= min_freq:
    vocab[tok] = len(vocab)

vocab['<unk>'] = len(vocab)
vocab['<pad>'] = len(vocab)

In [ ]:
for idx, val in enumerate(X_train_tokens):
  X_train_tokens[idx] = [vocab[i] if i in vocab else vocab['<unk>'] for i in val]
  if len(X_train_tokens[idx]) < max_size:
    X_train_tokens[idx].extend([vocab["<pad>"] for i in range(max_size - len(X_train_tokens[idx]))])
  if len(X_train_tokens[idx]) > max_size:
    X_train_tokens[idx] = X_train_tokens[:max_size]

for idx, val in enumerate(X_test_tokens):
  X_test_tokens[idx] = [vocab[i] if i in vocab else vocab['<unk>'] for i in val]
  if len(X_test_tokens[idx]) < max_size:
      X_test_tokens[idx].extend([vocab["<pad>"] for i in range(max_size - len(X_test_tokens[idx]))])
  if len(X_test_tokens[idx]) > max_size:
    X_test_tokens[idx] = X_test_tokens[:max_size]



for idx, val in enumerate(X_val_tokens):
  X_val_tokens[idx] = [vocab[i] if i in vocab else vocab['<unk>'] for i in val]
  if len(X_val_tokens[idx]) < max_size:
      X_val_tokens[idx].extend([vocab["<pad>"] for i in range(max_size - len(X_val_tokens[idx]))])
  if len(X_val_tokens[idx]) > max_size:
    X_val_tokens[idx] = X_val_tokens[:max_size]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = torch.tensor(self.texts[idx])
        label = torch.tensor(self.labels[idx])
        return text, label

In [ ]:
device = 'cuda:0'

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, hidden_dim, embedding_dim, num_layers, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(len(vocab), embedding_dim, padding_idx = vocab['<pad>'])
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc_1 = nn.Linear(hidden_dim, output_dim)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).to(device)
        out, _  = self.lstm(x,(h0,c0))
        out = self.fc(out[:, -1, :])
        out = self.tanh(out)
        out = self.fc_1(out)
        return out

In [ ]:
batch_size = 256

train_dataset = CustomDataset(X_train_tokens, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(X_val_tokens, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = CustomDataset(X_test_tokens, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
model = BiLSTM(hidden_dim=max_size, embedding_dim=512, num_layers=2, output_dim=10)
model.to(device)

BiLSTM(
  (embedding): Embedding(12910, 512, padding_idx=12909)
  (lstm): LSTM(512, 110, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=220, out_features=110, bias=True)
  (fc_1): Linear(in_features=110, out_features=10, bias=True)
  (tanh): Tanh()
)

In [ ]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.000001)

In [ ]:
def train_model():
  threshold = 0.5

  total_loss = 0
  total_samples = 0
  total_correct = 0
  model.train()

  for idx, batch in enumerate(train_dataloader):
    input_ids, labels = batch
    input_ids = input_ids.to(device, dtype=torch.int64)
    labels = (labels > 0).to(device, dtype=torch.float)

    outputs = model(input_ids)
    outputs = torch.sigmoid(outputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss.item()

    predictions = outputs >= threshold
    correct_predictions = (predictions == labels).all(dim=1)
    total_correct += correct_predictions.sum().item()
    total_samples += labels.size(0)

  avg_loss = total_loss / len(train_dataloader)
  accuracy = total_correct / total_samples

  print(f'Train Loss: {avg_loss}, Train Accuracy: {accuracy}')

  total_loss = 0
  total_samples = 0
  total_correct = 0
  model.eval()

  for idx, batch in enumerate(val_dataloader):
    input_ids, labels = batch
    input_ids = input_ids.to(device, dtype=torch.int64)
    labels = (labels > 0).to(device, dtype=torch.float)

    outputs = model(input_ids)
    outputs = torch.sigmoid(outputs)
    loss = criterion(outputs, labels)
    total_loss += loss.item()

    predictions = outputs >= threshold
    correct_predictions = (predictions == labels).all(dim=1)
    total_correct += correct_predictions.sum().item()
    total_samples += labels.size(0)

  avg_loss = total_loss / len(val_dataloader)
  accuracy = total_correct / total_samples

  print(f'Valid Loss: {avg_loss}, Valid Accuracy: {accuracy}')

In [ ]:
# OUTPUT OF THIS CELL IS STORED MANUALLY IN THE NEXT CELL FOR FUTURE USE
for i in range(0, 100):
  print(i + 1)
  train_model()
  torch.save(model.state_dict(), f'/content/drive/Shareddrives/TFQ/SentNoB_After_Def/LSTM_CheckPoints/LSTM_{i+1}')

1
Train Loss: 0.6958995461463928, Train Accuracy: 0.0
Valid Loss: 0.6964995165665945, Valid Accuracy: 0.0
2
Train Loss: 0.6955522420454998, Train Accuracy: 0.0
Valid Loss: 0.6961600085099539, Valid Accuracy: 0.0
3
Train Loss: 0.6951973109829183, Train Accuracy: 0.0
Valid Loss: 0.6958190898100535, Valid Accuracy: 0.0
4
Train Loss: 0.6948307387682856, Train Accuracy: 0.0
Valid Loss: 0.6954763333002726, Valid Accuracy: 0.0
5
Train Loss: 0.6944720015233877, Train Accuracy: 0.0
Valid Loss: 0.6951333383719126, Valid Accuracy: 0.0
6
Train Loss: 0.6941069826787832, Train Accuracy: 0.0
Valid Loss: 0.694788247346878, Valid Accuracy: 0.0
7
Train Loss: 0.6937657010798551, Train Accuracy: 0.0
Valid Loss: 0.6944423715273539, Valid Accuracy: 0.0
8
Train Loss: 0.6934127503511857, Train Accuracy: 0.0
Valid Loss: 0.6940921545028687, Valid Accuracy: 0.0
9
Train Loss: 0.6930630730122936, Train Accuracy: 0.0
Valid Loss: 0.6937404970328013, Valid Accuracy: 0.0
10
Train Loss: 0.6926644298495078, Train Accura

In [ ]:
for i in range(100, 300):
  print(i + 1)
  train_model()
  torch.save(model.state_dict(), f'/content/drive/Shareddrives/TFQ/SentNoB_After_Def/LSTM_CheckPoints/LSTM_{i+1}')

101
Train Loss: 0.5643958072273099, Train Accuracy: 0.0809342308004217
Valid Loss: 0.5701004564762115, Valid Accuracy: 0.038539553752535496
102
Train Loss: 0.5592310769217355, Train Accuracy: 0.0809342308004217
Valid Loss: 0.5653784473737081, Valid Accuracy: 0.038539553752535496
103
Train Loss: 0.5540485929469673, Train Accuracy: 0.0809342308004217
Valid Loss: 0.5606463948885599, Valid Accuracy: 0.038539553752535496
104
Train Loss: 0.5489253389592074, Train Accuracy: 0.0809342308004217
Valid Loss: 0.5559433897336324, Valid Accuracy: 0.038539553752535496
105
Train Loss: 0.5436202640436134, Train Accuracy: 0.0809342308004217
Valid Loss: 0.5513036747773489, Valid Accuracy: 0.038539553752535496
106
Train Loss: 0.5386511513165065, Train Accuracy: 0.0809342308004217
Valid Loss: 0.5467311044534048, Valid Accuracy: 0.038539553752535496
107
Train Loss: 0.5336474387013183, Train Accuracy: 0.0809342308004217
Valid Loss: 0.5422804554303488, Valid Accuracy: 0.038539553752535496
108
Train Loss: 0.52

KeyboardInterrupt: ignored

In [ ]:
def test_model():
  model.eval()

  all_predictions = []
  all_labels = []

  total_correct = 0
  total_samples = 0

  for idx, batch in enumerate(test_dataloader):
    input_ids, labels = batch
    input_ids = input_ids.to(device, dtype=torch.int64)
    labels = (labels > 0).to(device, dtype=torch.float)

    outputs = model(input_ids)
    outputs = torch.sigmoid(outputs)

    predictions = outputs >= 0.5

    correct_predictions = (predictions == labels).all(dim=1)
    total_correct += correct_predictions.sum().item()
    total_samples += labels.size(0)

    all_predictions.append(predictions.cpu().numpy())
    all_labels.append(labels.cpu().numpy())

  all_predictions_np = np.concatenate([p for p in all_predictions])
  all_labels_np = np.concatenate([l for l in all_labels])

  accuracy = total_correct / total_samples
  precision = precision_score(all_labels_np, all_predictions_np, average='micro')
  recall = recall_score(all_labels_np, all_predictions_np, average='micro')
  f1 = f1_score(all_labels_np, all_predictions_np, average='micro')

  print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')
  return all_predictions_np, all_labels_np

In [ ]:
model.load_state_dict(torch.load(f'/content/drive/Shareddrives/TFQ/SentNoB_After_Def/LSTM_CheckPoints/LSTM_82'))
y_pred, y_true = test_model()

Accuracy: 0.06123822341857335, Precision: 0.3620457604306864, Recall: 0.18430969510106202, F1-Score: 0.2442678774120318


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       127
           1       0.00      0.00      0.00       152
           2       0.00      0.00      0.00        82
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00         4
           5       0.36      1.00      0.53       538
           6       0.00      0.00      0.00       881
           7       0.00      0.00      0.00       429
           8       0.00      0.00      0.00       609
           9       0.00      0.00      0.00        47

   micro avg       0.36      0.18      0.24      2919
   macro avg       0.04      0.10      0.05      2919
weighted avg       0.07      0.18      0.10      2919
 samples avg       0.36      0.18      0.23      2919



In [ ]:
multilabel_confusion_matrix(y_true, y_pred)

array([[[1359,    0],
        [ 127,    0]],

       [[1334,    0],
        [ 152,    0]],

       [[1404,    0],
        [  82,    0]],

       [[1436,    0],
        [  50,    0]],

       [[1482,    0],
        [   4,    0]],

       [[   0,  948],
        [   0,  538]],

       [[ 605,    0],
        [ 881,    0]],

       [[1057,    0],
        [ 429,    0]],

       [[ 877,    0],
        [ 609,    0]],

       [[1439,    0],
        [  47,    0]]])